# Pakistan vs Neighbors — Internet Connectivity analysis

This notebook loads the ITU/World Bank export, filters and cleans records for Pakistan and comparator countries (Bangladesh, India, China, Russia, Afghanistan), computes a simple global mean (unweighted), saves a Kaggle-ready CSV, and runs EDA + statistical comparisons with Plotly visualizations.

Run the first code cell to create `kaggle_ready_internet_connectivity_pak_vs_neighbors.csv`.

In [1]:
# Imports
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from scipy import stats
import os

pd.set_option('display.max_rows', 150)

In [2]:
import os
import pandas as pd

# Use workspace absolute path to ensure notebook finds the file on Windows
raw_path = '/kaggle/input/household-internet-connectivity-itu-world-bank/kaggle_ready_internet_connectivity_pak_vs_neighbors.csv'

if not os.path.exists(raw_path):
    print(f'Raw file {raw_path} not found in workspace — ensure it is present.')
else:
    df_raw = pd.read_csv(raw_path, encoding='latin-1')
    print("Data loaded successfully!")
    print(df_raw.head())


Data loaded successfully!
      Country ISO_Code  Year  Connectivity_Rate_Percent
0  Bangladesh      BGD  2013                   2.685530
1  Bangladesh      BGD  2021                   0.798809
2       China      MAC  2007                  75.221200
3    Pakistan      PAK  2016                   4.475010
4    Pakistan      PAK  2020                   1.402370


In [3]:
import os
import pandas as pd

# Path to dataset
raw_path = '/kaggle/input/household-internet-connectivity-itu-world-bank/kaggle_ready_internet_connectivity_pak_vs_neighbors.csv'

# Check file exists
if not os.path.exists(raw_path):
    raise FileNotFoundError(f'Raw file {raw_path} not found in workspace — ensure it is present.')

# Load dataset
df_raw = pd.read_csv(raw_path, encoding='latin-1')

# Inspect columns
print("Available columns:", df_raw.columns.tolist())

# Define mapping based on actual column names
# Adjust these keys after seeing the print output above
cols_map = {
    "Country Name": "Country",
    "Year": "Year",
    "Connectivity (%)": "Internet_Connectivity"
}

# Validate mapping
missing = [k for k in cols_map.keys() if k not in df_raw.columns]
if missing:
    print("⚠️ Missing columns in raw file:", missing)
else:
    df = df_raw[list(cols_map.keys())].rename(columns=cols_map).copy()
    print("✅ Clean dataframe ready!")
    print(df.head())


Available columns: ['Country', 'ISO_Code', 'Year', 'Connectivity_Rate_Percent']
⚠️ Missing columns in raw file: ['Country Name', 'Connectivity (%)']


In [4]:
import os
import pandas as pd

# Path to dataset
raw_path = '/kaggle/input/household-internet-connectivity-itu-world-bank/kaggle_ready_internet_connectivity_pak_vs_neighbors.csv'

# Load dataset
df_raw = pd.read_csv(raw_path, encoding='latin-1')

# Show actual column names
print("Available columns in raw file:", df_raw.columns.tolist())

# Example mapping (⚠️ adjust keys after checking the print output above)
# Replace the keys here with the exact names printed above
cols_map = {
    "REF_AREA": "Country",
    "TIME_PERIOD": "Year",
    "OBS_VALUE": "Internet_Connectivity"
}

# Build clean dataframe only if all keys exist
missing = [k for k in cols_map.keys() if k not in df_raw.columns]
if missing:
    print("⚠️ Missing columns in raw file:", missing)
    # Fallback: just copy the raw dataframe for now
    df = df_raw.copy()
else:
    df = df_raw[list(cols_map.keys())].rename(columns=cols_map).copy()

print("✅ Clean dataframe ready!")
print(df.head())


Available columns in raw file: ['Country', 'ISO_Code', 'Year', 'Connectivity_Rate_Percent']
⚠️ Missing columns in raw file: ['REF_AREA', 'TIME_PERIOD', 'OBS_VALUE']
✅ Clean dataframe ready!
      Country ISO_Code  Year  Connectivity_Rate_Percent
0  Bangladesh      BGD  2013                   2.685530
1  Bangladesh      BGD  2021                   0.798809
2       China      MAC  2007                  75.221200
3    Pakistan      PAK  2016                   4.475010
4    Pakistan      PAK  2020                   1.402370


In [5]:
import os
import pandas as pd

# Path to dataset
raw_path = '/kaggle/input/household-internet-connectivity-itu-world-bank/kaggle_ready_internet_connectivity_pak_vs_neighbors.csv'

# Load dataset
df_raw = pd.read_csv(raw_path, encoding='latin-1')

# Show actual column names
print("Available columns in raw file:", df_raw.columns.tolist())

# ✅ Adjust mapping based on actual headers printed above
# Replace the keys here with the exact names from your dataset
cols_map = {
    "Country Name": "Country",
    "ISO_Code": "ISO_Code",
    "Year": "Year",
    "Connectivity_Rate_Percent": "Connectivity_Rate_Percent"
}

# Build clean dataframe safely
expected_cols = list(cols_map.keys())
missing = [c for c in expected_cols if c not in df_raw.columns]

if missing:
    print("⚠️ Missing columns:", missing)
    df = df_raw.copy()
else:
    df = df_raw[expected_cols].rename(columns=cols_map).copy()

# Define target countries and flexible matching
targets = {
    'Pakistan': ['Pakistan'],
    'Bangladesh': ['Bangladesh'],
    'India': ['India'],
    'China': ['China','Peoples Republic of China','China, mainland'],
    'Russia': ['Russian Federation','Russia'],
    'Afghanistan': ['Afghanistan']
}

# Filter rows for target countries
mask = pd.Series(False, index=df.index)
for name, variants in targets.items():
    for v in variants:
        mask = mask | df['Country'].str.contains(v, case=False, na=False)

df_targets = df[mask].copy()

# Normalize country names
for canonical, variants in targets.items():
    for v in variants:
        df_targets.loc[
            df_targets['Country'].str.contains(v, case=False, na=False),
            'Country'
        ] = canonical

# Compute global mean by year
df_global = df.groupby('Year', as_index=False)['Connectivity_Rate_Percent'].mean().copy()
df_global['Country'] = 'Global_Mean'
df_global['ISO_Code'] = 'GLOBAL'

# Combine targets + global mean
df_save = pd.concat([df_targets, df_global], ignore_index=True, sort=False)

# Reorder columns and drop missing
df_save = df_save[['Country','ISO_Code','Year','Connectivity_Rate_Percent']].dropna(subset=['Year','Connectivity_Rate_Percent']).copy()

# Save Kaggle-ready CSV
out_csv = 'kaggle_ready_internet_connectivity_pak_vs_neighbors.csv'
df_save.to_csv(out_csv, index=False)
print(f'✅ Saved {out_csv} — {len(df_save)} rows')
print(df_save.head())


Available columns in raw file: ['Country', 'ISO_Code', 'Year', 'Connectivity_Rate_Percent']
⚠️ Missing columns: ['Country Name']
✅ Saved kaggle_ready_internet_connectivity_pak_vs_neighbors.csv — 34 rows
      Country ISO_Code  Year  Connectivity_Rate_Percent
0  Bangladesh      BGD  2013                   2.685530
1  Bangladesh      BGD  2021                   0.798809
2       China      MAC  2007                  75.221200
3    Pakistan      PAK  2016                   4.475010
4    Pakistan      PAK  2020                   1.402370


In [6]:

# Quick EDA: summary table
summary = df_save.groupby('Country')['Connectivity_Rate_Percent'].describe()
display(summary)

# Timeseries plot — interactive with Plotly
fig = px.line(
    df_save.sort_values(['Country','Year']),
    x='Year',
    y='Connectivity_Rate_Percent',
    color='Country',
    markers=True,
    title='Internet Connectivity Rate (%) — Pakistan vs Neighbors + Global Mean'
)

# Update layout for readability
fig.update_layout(
    yaxis_title='Connectivity Rate (%)',
    xaxis=dict(dtick=1)
)

# Ensure plots folder exists and save
os.makedirs('plots', exist_ok=True)
fig.write_html('plots/connectivity_timeseries.html', include_plotlyjs='cdn')

# Show plot inline
fig


,count,mean,std,min,25%,50%,75%,max
Country,,,,,,,,
Bangladesh,2.0,1.742169,1.334113,0.798809,1.270489,1.742169,2.213850,2.685530
China,5.0,54.944240,14.382901,40.500000,45.800000,48.900000,64.300000,75.221200
Global_Mean,25.0,32.375839,11.114484,16.253401,21.850955,34.428449,39.514662,57.299644
Pakistan,2.0,2.938690,2.172685,1.402370,2.170530,2.938690,3.706850,4.475010


In [7]:
import pandas as pd
from scipy import stats  # ✅ import scipy.stats

# Pivot to wide format: years as index, countries as columns
wide = df_save.pivot_table(index='Year', columns='Country', values='Connectivity_Rate_Percent')

# Extract Pakistan series if available
pak = wide['Pakistan'] if 'Pakistan' in wide.columns else None

results = []
for c in ['India','China','Bangladesh','Russia','Afghanistan']:
    if c not in wide.columns or pak is None:
        continue
    # Keep overlapping years only
    pair = wide[[c,'Pakistan']].dropna()
    if len(pair) < 3:
        # Not enough overlapping years for a meaningful test
        results.append((c, len(pair), None, None))
        continue
    # Welch’s t-test (unequal variances)
    tstat, pval = stats.ttest_ind(pair['Pakistan'], pair[c], equal_var=False, nan_policy='omit')
    results.append((c, len(pair), tstat, pval))

# Build results dataframe
res_df = pd.DataFrame(results, columns=['Country','N_overlapping_years','t_stat','p_value']).set_index('Country')

# Display results
display(res_df)

# Save results to CSV
res_df.to_csv('connectivity_ttest_results.csv')
print('✅ Saved connectivity_ttest_results.csv')


,N_overlapping_years,t_stat,p_value
Country,,,
China,1,None,None
Bangladesh,0,None,None


✅ Saved connectivity_ttest_results.csv


### Notes:
- The notebook saved `kaggle_ready_internet_connectivity_pak_vs_neighbors.csv` and `plots/connectivity_timeseries.html`.
- `connectivity_ttest_results.csv` contains basic two-sample t-test comparisons (Pakistan vs each neighbor) using available overlapping years.
- Adjust the year range or statistical method as needed for more rigorous analysis.